In [ ]:
#Clone Repo
%cd /content
!git clone https://github.com/rse4399/DataScience_Project
%cd DataScience_Project/data

/content
Cloning into 'DataScience_Project'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 16 (delta 1), reused 16 (delta 1), pack-reused 0
Unpacking objects: 100% (16/16), done.
/content/DataScience_Project/data


In [ ]:
#Import Libraries
import numpy as np
import pandas as pd
import glob2
import urllib
import urllib.request
import zipfile

In [ ]:
#NOTE: This only works in Google Collab Pro due to RAM limitations
#Download all 2019 Citibike Data. Extracts csv(s) into /2019/

urls_2019 = ["https://s3.amazonaws.com/tripdata/201901-citibike-tripdata.csv.zip",
             "https://s3.amazonaws.com/tripdata/201902-citibike-tripdata.csv.zip",
             "https://s3.amazonaws.com/tripdata/201903-citibike-tripdata.csv.zip",
             "https://s3.amazonaws.com/tripdata/201904-citibike-tripdata.csv.zip",
             "https://s3.amazonaws.com/tripdata/201905-citibike-tripdata.csv.zip",
             "https://s3.amazonaws.com/tripdata/201906-citibike-tripdata.csv.zip",
             "https://s3.amazonaws.com/tripdata/201907-citibike-tripdata.csv.zip",
             "https://s3.amazonaws.com/tripdata/201908-citibike-tripdata.csv.zip",
             "https://s3.amazonaws.com/tripdata/201909-citibike-tripdata.csv.zip",
             "https://s3.amazonaws.com/tripdata/201910-citibike-tripdata.csv.zip",
             "https://s3.amazonaws.com/tripdata/201911-citibike-tripdata.csv.zip",
             "https://s3.amazonaws.com/tripdata/201912-citibike-tripdata.csv.zip",]

for url in urls_2019:
  extract_dir = "2019"
  zip_path, _ = urllib.request.urlretrieve(url)
  with zipfile.ZipFile(zip_path, "r") as f:
      f.extractall(extract_dir)

In [ ]:
#Read all csv into single dataframe
all_files = glob2.glob("./2019/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

df = pd.concat(li, axis=0, ignore_index=True)

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
#Drop unused columns and Na Columns
df=df.drop(columns="birth year")
df=df.drop(columns="gender")
df=df.drop(columns="usertype")
df=df.drop(columns="bikeid")
df=df.drop(columns="end station name")
df=df.drop(columns="start station name")

In [ ]:
#Get all unique start and end stations. Including Lat and Long for conversion
df_start_loc = df[['start station id','start station latitude','start station longitude']].groupby(['start station id','start station latitude','start station longitude']).size().reset_index(name='start_counts')
df_stop_loc = df[['end station id','end station latitude','end station longitude']].groupby(['end station id','end station latitude','end station longitude']).size().reset_index(name='end_counts')
df_start_loc.rename(columns={"start station id": "station_id"}, inplace=True)
df_stop_loc.rename(columns={"end station id": "station_id"}, inplace=True)
df_loc = pd.merge(df_start_loc, df_stop_loc,  how='left', left_on=['station_id'], right_on = ['station_id'])
df_loc=df_loc.drop(['start_counts','end_counts','end station latitude','end station longitude'],axis="columns")
df_loc.rename(columns={"start station latitude": "lat"}, inplace=True)
df_loc.rename(columns={"start station longitude": "lon"}, inplace=True)

station_ids = df["start station id"].unique()

df_loc= df_loc.drop_duplicates()
df_loc

,station_id,lat,lon
0,72.0,40.767272,-73.993929
1,79.0,40.719116,-74.006667
2,82.0,40.711174,-74.000165
3,83.0,40.683826,-73.976323
4,116.0,40.741776,-74.001497
...,...,...,...
1106,3907.0,40.701120,-73.930390
1107,3908.0,40.704630,-74.015070
1108,3909.0,40.704080,-73.904080
1109,3910.0,40.709697,-73.907856


In [ ]:
#Converts Lat/Lon to ZipCode
from geopy.geocoders import Nominatim

zip_codes = []

for index, row in df_loc.iterrows():
  lat_lon = row["lat"].astype(str)+", "+row["lon"].astype(str)
  #print(lat_lon)
  geolocator = Nominatim(user_agent="geoapiExercises")
  location = geolocator.geocode(lat_lon)
  data = location.raw
  loc_data = data['display_name'].split()
  zip = loc_data[-3][:-1]
  zip_codes.append(zip)

df_loc = df_loc.assign(zipcode=zip_codes)
df_loc.to_csv("station_to_zip.csv", encoding='utf-8', index=False)

#Clean up CSV manually in Excel for any errors

In [ ]:
#convert ZipCode to NYC Neighborhood

zip_neigh = "nyc_zip_borough_neighborhoods_pop.csv"
station_zip = "station_to_zip_fixed.csv"

df_neighborhood = pd.read_csv(zip_neigh, index_col=None, header=0)
df_neighborhood.rename(columns={"zip": "zipcode"}, inplace=True)
df_neighborhood=df_neighborhood.drop(columns="post_office")
df_neighborhood=df_neighborhood.drop(columns="borough")
df_neighborhood=df_neighborhood.drop(columns="population")
df_neighborhood=df_neighborhood.drop(columns="density")


df_station_zip = pd.read_csv(station_zip, index_col=None, header=0)
df_station_zip=df_station_zip.drop(columns="lat")
df_station_zip=df_station_zip.drop(columns="lon")

df_station_neighborhood = pd.merge(df_station_zip, df_neighborhood,  how='left', left_on=['zipcode'], right_on = ['zipcode'])
df_station_neighborhood.dropna(subset = ["neighborhood"], inplace=True)
df_station_neighborhood=df_station_neighborhood.drop(columns="zipcode")
df_station_neighborhood.to_csv("station_to_neighborhood.csv", encoding='utf-8', index=False)

In [ ]:
#Parsing out Start and stop  Time into Year, Month, Day, Hour
df["starttime"] = pd.to_datetime(df["starttime"])
df["Start_Year"] = pd.DatetimeIndex(df["starttime"]).year
df["Start_Month"] = pd.DatetimeIndex(df["starttime"]).month
df["Start_Day"] = pd.DatetimeIndex(df["starttime"]).day
df["Start_Hour"] = pd.DatetimeIndex(df["starttime"]).hour
df=df.drop(columns="starttime")

In [ ]:
#Parsing out  stop  Time into Year, Month, Day, Hour
df["stoptime"] = pd.to_datetime(df["stoptime"])
df["Stop_Year"] = pd.DatetimeIndex(df["stoptime"]).year
df["Stop_Month"] = pd.DatetimeIndex(df["stoptime"]).month
df["Stop_Day"] = pd.DatetimeIndex(df["stoptime"]).day
df["Stop_Hour"] = pd.DatetimeIndex(df["stoptime"]).hour
df=df.drop(columns="stoptime")

In [ ]:
#Load saved neighboorhood data

station_neighborhood = "station_to_neighborhood.csv"

df_station_neighborhood = pd.read_csv(station_neighborhood, index_col=None, header=0)

In [ ]:
#Get neighboorhood station count
df_neighborhood_count = df_station_neighborhood['neighborhood'].value_counts()
df_neighborhood_count.to_csv("neighborhood_station_count.csv", encoding='utf-8', index=False)

In [ ]:
#Add Start Neighboorhood data
df_station_neighborhood.rename(columns={"station_id": "start station id"}, inplace=True)
df_station_neighborhood.rename(columns={"neighborhood": "start_nh"}, inplace=True)
df = pd.merge(df, df_station_neighborhood,  how='left', left_on=['start station id'], right_on = ['start station id'])

In [ ]:
#Add End Neighboorhood data
df_station_neighborhood = pd.read_csv(station_neighborhood, index_col=None, header=0)
df_station_neighborhood.rename(columns={"station_id": "end station id"}, inplace=True)
df_station_neighborhood.rename(columns={"neighborhood": "end_nh"}, inplace=True)
df = pd.merge(df, df_station_neighborhood,  how='left', left_on=['end station id'], right_on = ['end station id'])

In [ ]:
#Get Outgoing Demand
df_start_count = df[['start_nh','Start_Year','Start_Month','Start_Day','Start_Hour']].groupby(["start_nh","Start_Year","Start_Month","Start_Day","Start_Hour"]).size().reset_index(name='start_counts')
df_start_count

,start_nh,Start_Year,Start_Month,Start_Day,Start_Hour,start_counts
0,Bushwick and Williamsburg,2019,1,1,0,1
1,Bushwick and Williamsburg,2019,1,1,1,6
2,Bushwick and Williamsburg,2019,1,1,2,8
3,Bushwick and Williamsburg,2019,1,1,3,5
4,Bushwick and Williamsburg,2019,1,1,4,2
...,...,...,...,...,...,...
187863,West Side,2019,12,31,19,4
187864,West Side,2019,12,31,20,1
187865,West Side,2019,12,31,21,2
187866,West Side,2019,12,31,22,2


In [ ]:
#Get Incoming Demand
df_stop_count = df[['end_nh','Stop_Year','Stop_Month','Stop_Day','Stop_Hour']].groupby(['end_nh','Stop_Year','Stop_Month','Stop_Day','Stop_Hour']).size().reset_index(name='stop_counts')
df_stop_count

,end_nh,Stop_Year,Stop_Month,Stop_Day,Stop_Hour,stop_counts
0,Bushwick and Williamsburg,2019,1,1,0,1
1,Bushwick and Williamsburg,2019,1,1,1,4
2,Bushwick and Williamsburg,2019,1,1,2,9
3,Bushwick and Williamsburg,2019,1,1,3,4
4,Bushwick and Williamsburg,2019,1,1,4,6
...,...,...,...,...,...,...
189111,West Side,2019,12,31,18,8
189112,West Side,2019,12,31,19,7
189113,West Side,2019,12,31,20,1
189114,West Side,2019,12,31,21,2


In [ ]:
#Rename to merge outgoing and incoming demand
df_start_count.rename(columns={"start_nh": "neighborhood"}, inplace=True)
df_start_count.rename(columns={"Start_Year": "Year"}, inplace=True)
df_start_count.rename(columns={"Start_Month": "Month"}, inplace=True)
df_start_count.rename(columns={"Start_Day": "Day"}, inplace=True)
df_start_count.rename(columns={"Start_Hour": "Hour"}, inplace=True)
df_stop_count.rename(columns={"end_nh": "neighborhood"}, inplace=True)
df_stop_count.rename(columns={"Stop_Year": "Year"}, inplace=True)
df_stop_count.rename(columns={"Stop_Month": "Month"}, inplace=True)
df_stop_count.rename(columns={"Stop_Day": "Day"}, inplace=True)
df_stop_count.rename(columns={"Stop_Hour": "Hour"}, inplace=True)

In [ ]:
#Merge start and stop dataframes using neighborhood
df_demand = pd.merge(df_start_count, df_stop_count,  how='left', left_on=['neighborhood','Year','Month','Day','Hour'], right_on = ['neighborhood','Year','Month','Day','Hour'])
df_demand['stop_counts'] = df_demand['stop_counts'].fillna(0)
df_demand['start_counts'] = df_demand['start_counts'].fillna(0)
df_demand

,neighborhood,Year,Month,Day,Hour,start_counts,stop_counts
0,Bushwick and Williamsburg,2019,1,1,0,1,1.0
1,Bushwick and Williamsburg,2019,1,1,1,6,4.0
2,Bushwick and Williamsburg,2019,1,1,2,8,9.0
3,Bushwick and Williamsburg,2019,1,1,3,5,4.0
4,Bushwick and Williamsburg,2019,1,1,4,2,6.0
...,...,...,...,...,...,...,...
187863,West Side,2019,12,31,19,4,7.0
187864,West Side,2019,12,31,20,1,1.0
187865,West Side,2019,12,31,21,2,2.0
187866,West Side,2019,12,31,22,2,0.0


In [ ]:
#Calculate demand using outgoing and incoming
df_demand["demand"] = df_demand["start_counts"]-df_demand["stop_counts"]
df_demand

,neighborhood,Year,Month,Day,Hour,start_counts,stop_counts,demand
0,Bushwick and Williamsburg,2019,1,1,0,1,1.0,0.0
1,Bushwick and Williamsburg,2019,1,1,1,6,4.0,2.0
2,Bushwick and Williamsburg,2019,1,1,2,8,9.0,-1.0
3,Bushwick and Williamsburg,2019,1,1,3,5,4.0,1.0
4,Bushwick and Williamsburg,2019,1,1,4,2,6.0,-4.0
...,...,...,...,...,...,...,...,...
187863,West Side,2019,12,31,19,4,7.0,-3.0
187864,West Side,2019,12,31,20,1,1.0,0.0
187865,West Side,2019,12,31,21,2,2.0,0.0
187866,West Side,2019,12,31,22,2,0.0,2.0


In [ ]:
#OPTIONAL
#Save 2019 prepared data locally
df_demand.to_csv("ds_citibike_2019_cleaned.csv", encoding='utf-8', index=False)

In [ ]:
#Get current Dock Information
import urllib
import urllib.request
import json

jsonurl = urllib.request.urlopen('https://gbfs.citibikenyc.com/gbfs/en/station_status.json')
text = json.loads(jsonurl.read()) 
data=text['data']
data

# station id Extract:
total_stations_id = []
for i in range(len(data['stations'])):
    id=data['stations'][i]['station_id']
    total_stations_id.append(id)
total_stations_id=[int(x) for x in total_stations_id]

# num_bikes_available Extract:
total_num_bikes_available = []
for i in range(len(data['stations'])):
    num_bikes_available = data['stations'][i]['num_bikes_available']
    total_num_bikes_available.append(num_bikes_available)

# num_docks_available Extract:
total_docks_available = []
for i in range(len(data['stations'])):
    docks_availble=data['stations'][i]['num_docks_available']
    total_docks_available.append(docks_availble)

total_docks=[x + y for x, y in zip(total_num_bikes_available, total_docks_available)]

station_dock = pd.DataFrame()
station_dock["station_id"] = total_stations_id
station_dock["docks"] = total_docks


In [ ]:
#OPTIONAL: Save staiton_dock to csv
station_dock.to_csv("station_dock.csv", encoding='utf-8', index=False)

In [ ]:
#START HERE IF REPO IS CLONED
citibike_2019 = "ds_citibike_2019_cleaned.csv"
neightbor_station = "neighborhood_station_count.csv"
station_to_neighborhood = "station_to_neighborhood.csv"
station_dock = "station_dock.csv"
weather = "weather_2019.csv"

df_citibike_2019 = pd.read_csv(citibike_2019, index_col=None, header=0)
df_neightbor_station_count = pd.read_csv(neightbor_station, index_col=None, header=0)
df_station_to_neighborhood = pd.read_csv(station_to_neighborhood, index_col=None, header=0)
df_station_dock = pd.read_csv(station_dock, index_col=None, header=0)
df_weather = pd.read_csv(weather, index_col=None, header=0)

In [ ]:
df = pd.merge(df_station_to_neighborhood, df_station_dock,  how='left', left_on=['station_id'], right_on = ['station_id'])
df = df.groupby(['neighborhood']).sum()
df=df.drop(columns="station_id")
df["percent"] = df["docks"]/df["docks"].sum()
df

,docks,percent
neighborhood,,
Bushwick and Williamsburg,2023.0,0.071862
Central Brooklyn,1195.0,0.042450
Central Harlem,1221.0,0.043373
Chelsea and Clinton,3503.0,0.124436
East Harlem,935.0,0.033214
East New York and New Lots,520.0,0.018472
Flatbush,182.0,0.006465
Gramercy Park and Murray Hill,1727.0,0.061348
Greenpoint,1651.0,0.058648


In [ ]:
#Add Dock percentage and dock count
df_citibike_2019 = pd.merge(df_citibike_2019, df,  how='left', left_on=['neighborhood'], right_on = ['neighborhood'])
df_citibike_2019

,neighborhood,Year,Month,Day,Hour,start_counts,stop_counts,demand,docks,percent
0,Bushwick and Williamsburg,2019,1,1,0,1,1.0,0.0,2023.0,0.071862
1,Bushwick and Williamsburg,2019,1,1,1,6,4.0,2.0,2023.0,0.071862
2,Bushwick and Williamsburg,2019,1,1,2,8,9.0,-1.0,2023.0,0.071862
3,Bushwick and Williamsburg,2019,1,1,3,5,4.0,1.0,2023.0,0.071862
4,Bushwick and Williamsburg,2019,1,1,4,2,6.0,-4.0,2023.0,0.071862
...,...,...,...,...,...,...,...,...,...,...
187863,West Side,2019,12,31,19,4,7.0,-3.0,157.0,0.005577
187864,West Side,2019,12,31,20,1,1.0,0.0,157.0,0.005577
187865,West Side,2019,12,31,21,2,2.0,0.0,157.0,0.005577
187866,West Side,2019,12,31,22,2,0.0,2.0,157.0,0.005577


In [ ]:
#Add Weather Data
#Source from https://www.visualcrossing.com/weather-data
df_citibike_2019 = pd.merge(df_citibike_2019, df_weather,  how='left', left_on=['Year','Month','Day','Hour'], right_on = ['Year','Month','Day','Hour'])

In [ ]:
#Add Risk assement
df_citibike_2019['risk'] = np.where(abs(df_citibike_2019['demand']) > (0.025*df_citibike_2019["docks"]) , 1, 0)

In [ ]:
#OPTIONAL:
df_citibike_2019.to_csv("ds_citibike_2019_final.csv", encoding='utf-8', index=False)